# Detection of Movie Spoilers in Reviews

##### ***Import essential libraries***

In [67]:
import pandas as pd
import numpy as np
import ast #to convert series of strings to series of lists

##### Understanding the data

Apories:
1. what is rating_x, rating_y
2. maybe convert duration to minutes would be better to compare

In [64]:
original_data = pd.read_csv('movie_spoiler_sample.csv')

In [13]:
original_data.head(5)

,movie_id,plot_summary,duration,genre,rating_x,release_date,plot_synopsis,review_date,user_id,is_spoiler,review_text,rating_y,review_summary
0,tt0318649,Master explorer and former US Navy Seal Dirk P...,2h 4min,"['Action', 'Adventure', 'Comedy']",6.0,2005-04-08,The film begins with a prologue set in Richmon...,27 May 2005,ur3270789,False,The film starts in the Richmond battle (1865) ...,7,Splendid adventure film with mesmerizing deser...
1,tt0803096,When the world of the Orcs of Draenor is being...,2h 3min,"['Action', 'Adventure', 'Fantasy']",6.9,2016-06-10,"For ages in the region of Middle Earth, humans...",20 June 2016,ur47788388,True,I saw the movie with six friends and we all lo...,10,Epic movie for fans and non fans
2,tt0425112,"Top London cop, PC Nicholas Angel is good. Too...",2h 1min,"['Action', 'Comedy']",7.9,2007-04-20,Nicholas Angel (Simon Pegg) is undoubtedly Lon...,11 June 2007,ur14440242,True,"I enjoyed this movie very much, but it being l...",8,once again a funny British film
3,tt0327056,In the summer of 1975 in a neighborhood in Bos...,2h 18min,"['Crime', 'Drama', 'Mystery']",8.0,2003-10-15,"Three young boys, all the best of friends, are...",5 November 2015,ur63623011,False,What you get here is no more than Clint eastwo...,1,For he made a 'plot twist' out of it
4,tt0259711,"Incarcerated and charged with murder, David Aa...",2h 16min,"['Fantasy', 'Mystery', 'Romance']",6.9,2001-12-14,"David Aames (Tom Cruise) drives to work, he fi...",12 August 2016,ur2781970,False,"I think the first time I saw this movie, I did...",10,As brilliant as I recalled it!


In [5]:
original_data.columns

Index(['movie_id', 'plot_summary', 'duration', 'genre', 'rating_x',
       'release_date', 'plot_synopsis', 'review_date', 'user_id', 'is_spoiler',
       'review_text', 'rating_y', 'review_summary'],
      dtype='object')

In [6]:
original_data.shape

(72054, 13)

In [10]:
original_data['is_spoiler'].value_counts() #it is balanced

is_spoiler
False    36277
True     35777
Name: count, dtype: int64

In [11]:
original_data.duplicated().any() #no duplicated rows

False

In [17]:
original_data.dtypes

movie_id           object
plot_summary       object
duration           object
genre              object
rating_x          float64
release_date       object
plot_synopsis      object
review_date        object
user_id            object
is_spoiler           bool
review_text        object
rating_y            int64
review_summary     object
dtype: object

In [60]:
original_data['review_date'][0]

'27 May 2005'

In [65]:
#remove starting and trailing white spaces from strings
columns_to_strip = ['movie_id','plot_summary','duration','genre','release_date','plot_synopsis',
                            'review_date','user_id','review_text','review_summary']
original_data[columns_to_strip] = original_data[columns_to_strip].apply(lambda col: col.str.strip())

In [72]:
#replace empty strings with NA
original_data[columns_to_strip] = original_data[columns_to_strip].replace('', np.nan)

In [73]:
#check how many null values each feature has
original_data.isnull().sum() #no missing values

movie_id          0
plot_summary      0
duration          0
genre             0
rating_x          0
release_date      0
plot_synopsis     0
review_date       0
user_id           0
is_spoiler        0
review_text       0
rating_y          0
review_summary    0
dtype: int64

In [18]:
genre = original_data['genre']
genre

0         ['Action', 'Adventure', 'Comedy']
1        ['Action', 'Adventure', 'Fantasy']
2                      ['Action', 'Comedy']
3             ['Crime', 'Drama', 'Mystery']
4         ['Fantasy', 'Mystery', 'Romance']
                        ...                
72049                            ['Comedy']
72050                 ['Comedy', 'Fantasy']
72051     ['Biography', 'Drama', 'History']
72052     ['Adventure', 'Comedy', 'Family']
72053     ['Action', 'Adventure', 'Sci-Fi']
Name: genre, Length: 72054, dtype: object

In [19]:
genre[0]

"['Action', 'Adventure', 'Comedy']"

In [33]:
#check if every element of genre feature is a string in order to make them lists
original_data['genre'].apply(lambda x: isinstance(x, str)).all()

True

In [34]:
#convert genre feature from strings to a lists
original_data['genre'] = original_data['genre'].apply(ast.literal_eval)

In [75]:
#convert duration to minutes
# Create a new column for duration in minutes
duration_minutes = []

for elem in original_data['duration']:
    # Initialize hours and minutes to 0
    hours = 0
    minutes = 0
    
    # Split the string by 'h'
    splitted = elem.split('h')
    
    # Check if the hour part exists
    if len(splitted) > 1:  # Contains both hours and minutes
        hours = int(splitted[0])
        minutes = int(splitted[1].split('min')[0].strip())
    elif 'h' in elem:  # Only hours
        hours = int(splitted[0].strip())
    elif 'min' in elem:  # Only minutes
        minutes = int(elem.split('min')[0].strip())
    
    # Calculate total time in minutes
    total_minutes = hours * 60 + minutes
    duration_minutes.append(total_minutes)

# Add the new column to the dataframe
original_data['duration_minutes'] = duration_minutes

ValueError: invalid literal for int() with base 10: ''

In [49]:
original_data['duration'][0].split('h ')[1]

'4min'